# LSTM examples

In [51]:
import pandas as pd
import numpy as np
from keras import Sequential, Model
from keras.layers import LSTM, Dense, Bidirectional, Input, TimeDistributed, RepeatVector
from keras.utils import get_file
import os

Load dataset

In [52]:
zip_path = get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)
df = pd.read_csv(csv_path)
df = df[5::6]
date_time = pd.to_datetime(df.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')
df.head()

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
5,996.50,-8.05,265.38,-8.78,94.4,3.33,3.14,0.19,1.96,3.15,1307.86,0.21,0.63,192.7
11,996.62,-8.88,264.54,-9.77,93.2,3.12,2.90,0.21,1.81,2.91,1312.25,0.25,0.63,190.3
17,996.84,-8.81,264.59,-9.66,93.5,3.13,2.93,0.20,1.83,2.94,1312.18,0.18,0.63,167.2
23,996.99,-9.05,264.34,-10.02,92.6,3.07,2.85,0.23,1.78,2.85,1313.61,0.10,0.38,240.0
29,997.46,-9.63,263.72,-10.65,92.2,2.94,2.71,0.23,1.69,2.71,1317.19,0.40,0.88,157.0


In [53]:
temperature_data = np.array(df['T (degC)'])

In [54]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(0, len(sequence)- n_steps):
		end_ix = i + n_steps
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	X = np.array(X)
	y = np.array(y)
	X = X.reshape(X.shape[0], n_steps, 1)
	return X, y

In [55]:
X, y = split_sequence(temperature_data, 3)
X.shape, y.shape

((70088, 3, 1), (70088,))

## Single step LSTM

In [56]:
model = Sequential([
    LSTM(50, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

In [57]:
model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10
2191/2191 [==============================] - 6s 2ms/step - loss: 3.3302
Epoch 2/10
2191/2191 [==============================] - 5s 2ms/step - loss: 0.6940
Epoch 3/10
2191/2191 [==============================] - 4s 2ms/step - loss: 0.6908
Epoch 4/10
2191/2191 [==============================] - 4s 2ms/step - loss: 0.6820
Epoch 5/10
2191/2191 [==============================] - 4s 2ms/step - loss: 0.6792
Epoch 6/10
2191/2191 [==============================] - 4s 2ms/step - loss: 0.6722
Epoch 7/10
2191/2191 [==============================] - 4s 2ms/step - loss: 0.6694
Epoch 8/10
2191/2191 [==============================] - 4s 2ms/step - loss: 0.6658
Epoch 9/10
2191/2191 [==============================] - 4s 2ms/step - loss: 0.6616
Epoch 10/10
2191/2191 [==============================] - 4s 2ms/step - loss: 0.6590


## Multi step LSTM

In [58]:
num_predictions = 2
model = Sequential([
    LSTM(50, activation='relu'),
    Dense(num_predictions)
])
model.compile(optimizer='adam', loss='mse')

## Multilayer LSTM

In [59]:
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True),
    LSTM(50, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

## Bidirectional LSTM

In [60]:
model = Sequential([
    Bidirectional(LSTM(units=50)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

In [61]:
model.fit(X, y, epochs=5, batch_size=32)

Epoch 1/5
2191/2191 [==============================] - 9s 2ms/step - loss: 5.3054
Epoch 2/5
2191/2191 [==============================] - 5s 2ms/step - loss: 0.7130
Epoch 3/5
2191/2191 [==============================] - 5s 2ms/step - loss: 0.6914
Epoch 4/5
2191/2191 [==============================] - 5s 2ms/step - loss: 0.6766
Epoch 5/5
2191/2191 [==============================] - 5s 2ms/step - loss: 0.6742


## LSTM Sequence to sequence (encoder decoder)

Model

In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Inference setup

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

Inference loop

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence